# Play Doom Level using Trained PPO Agent  

In [2]:
from utils.env import make_doom_env
from agents.doom_ppo_agent import DoomPpoAgent
import gym

# Creating Environment
num_envs = 1
envs = gym.vector.SyncVectorEnv([ make_doom_env(level_config_path='vizdoom/scenarios/basic.cfg', render=True) for i in range(num_envs)])
# Setting up agent
agent = DoomPpoAgent(envs.single_observation_space, 
                     envs.single_action_space, 
                     models_path='./models/doom_ppo_agent/training_run_2023_07_07_02_24_27/checkpoint_step_292864')

# Playing config
number_of_episodes = 100

Loading models...
Successfully loaded models!
Updating networks with weights from loaded models...
Successfully updated networks!


In [3]:
import time
import torch
import numpy as np

for episode in range(number_of_episodes):
    observation = envs.reset()
    done = np.zeros(num_envs, dtype=np.int32)
    total_reward = 0

    while not done.all():
        # Getting next action and it's value
        with torch.no_grad():
            action, log_prob, _, value = agent.get_optimal_action_and_value(torch.Tensor(observation).to(agent.device))
        observation, reward, done, info = envs.step(action.cpu().numpy())
        print(action.shape)
        total_reward += reward

        time.sleep(1/30)

    print(f"Total reward for episode {episode} is {total_reward}")
    time.sleep(2)

torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
torch.Size([1])
Total reward for episode 0 is [75.]
torch.Size([1])
torch.Size([1])
Total reward for episode 1 is [95.]


KeyboardInterrupt: 